The URL the user will go to give us info is below. This is just for beta and closed development.When the user gives us their info they will be met with localhost can't be connected to or something along the lines. For now the link is code= whatever in the url, you can copy and paste it. In the future when we get our own url, we will be able to have a function that just grabs the code without us interfering (callback function). The access token lasts for an hour and the auth code is one use. So if you used the auth code snippet once, please don't refresh it, it will give an error.

https://accounts.spotify.com/en/authorize?client_id=6b7e2e58b62d4010a958a1f14c3a2319&response_type=code&redirect_uri=http://localhost:8880/callback&scope=user-top-read%20user-read-private%20playlist-read-private

In [23]:
from flask import Flask,flash, render_template, redirect, session, make_response, request, url_for
from flask_sqlalchemy import SQLAlchemy
from urllib.parse import urlencode
import os
import requests
import logging
import numpy as np
import pandas as pd
import db
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from pathlib import Path
from datetime import datetime
import json
from sqlalchemy import PrimaryKeyConstraint


# Flask setup
app = Flask(__name__)
app.static_folder = 'static'
app.secret_key = os.urandom(24)
app.config.from_pyfile('config.py')

BASE_DIR = Path(os.getcwd())
DB_PATH = BASE_DIR / 'db' / 'User.db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{DB_PATH}'
app.config['SQLALCHEMY_BINDS'] = {
    'two': f'sqlite:///{BASE_DIR}/Playlist.db',
    'three': f'sqlite:///{BASE_DIR}/Song.db'
}

db = SQLAlchemy(app)
class User(db.Model):
    __tablename__ = 'user'  # Explicitly define the table name
    id = db.Column(db.String(255), primary_key=True)  # Spotify ID as primary key
    username = db.Column(db.String(80), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class Playlist(db.Model):
    __tablename__ = 'playlist'
    id = db.Column(db.String(255), primary_key=True, nullable=False)
    name = db.Column(db.String(255))
    user_id = db.Column(db.String(255), db.ForeignKey('user.id'), nullable=False)  # Changed to String to match User ID type
    user = db.relationship('User', backref=db.backref('playlists', lazy=True))


class Song(db.Model):
    __tablename__ = 'song'
    uid = db.Column(db.String(255), db.ForeignKey('user.id'), nullable=False)
    id = db.Column(db.String(255), nullable=False)
    name = db.Column(db.String(255))
    artists = db.Column(db.String(255))
    playlist_id = db.Column(db.String(255), db.ForeignKey('playlist.id'), nullable=True)
    playlist = db.relationship('Playlist', backref=db.backref('songs', lazy=True))
    top_tracks = db.Column(db.Boolean, default=None)

    __table_args__ = (
        PrimaryKeyConstraint('uid', 'id'),
        {},
    )
class SongCharacteristic(db.Model):
    __tablename__ = 'song_characteristic'  # Define a new table name for characteristics
    id = db.Column(db.String(255), db.ForeignKey('song.id'), primary_key=True)
    danceability = db.Column(db.Float)
    energy = db.Column(db.Float)
    key = db.Column(db.Integer)
    loudness = db.Column(db.Float)
    mode = db.Column(db.Integer)
    speechiness = db.Column(db.Float)
    acousticness = db.Column(db.Float)
    instrumentalness = db.Column(db.Float)
    liveness = db.Column(db.Float)
    valence = db.Column(db.Float)
    tempo = db.Column(db.Float)
    # Add other characteristics as needed

class Top_Artists(db.Model):
    __tablename__ = 'artist'
    uid = db.Column(db.String(255), primary_key=True)
    id = db.Column(db.String(255), primary_key=True, nullable=False)
    name = db.Column(db.String(255))


with app.app_context():
    db.create_all()
# Define constants
US_TOP_HITS_PLAYLIST_ID = "37i9dQZF1DXcBWIGoYBM5M?si=beaf719fd1f64d09"

def getToken(code):
    token_url = 'https://accounts.spotify.com/api/token'
    authorization = app.config['AUTHORIZATION']
    redirect_uri = app.config['REDIRECT_URI']
    headers = {'Authorization': f"Basic {authorization}",  # Updated the Authorization header
               'Accept': 'application/json',
               'Content-Type': 'application/x-www-form-urlencoded'}
    body = {'code': code, 'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'}
    post_response = requests.post(token_url, headers=headers, data=body)
    if post_response.status_code == 200:
        pr = post_response.json()
        return pr['access_token'], pr['refresh_token'], pr['expires_in']
    else:
        logging.error('getToken:' + str(post_response.status_code))
        return None
def get_top_tracks(user_id, access_token, limit=50):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/top/tracks?limit={limit}", headers=headers)
    
    if response.status_code == 200:
        spotify_tracks = response.json()['items']
        tracks_to_update_or_create = []

        # Start a database session context
        with db.session.begin():
            for track in spotify_tracks:
                track_id = track['id']
                # Check if the song is already in the database by filtering with user ID and track ID
                song = Song.query.filter_by(uid=user_id, id=track_id).first()
                
                if song:
                    # If the song exists, ensure top_tracks is True
                    if not song.top_tracks:
                        song.top_tracks = True
                        tracks_to_update_or_create.append(song)
                else:
                    # If the song does not exist, create a new record with top_tracks as True
                    song = Song(
                        uid=user_id,
                        id=track_id,
                        name=track['name'],
                        artists=', '.join(artist['name'] for artist in track['artists']),
                        top_tracks=True  # Set top_tracks to True for new songs
                    )
                    db.session.add(song)
                    tracks_to_update_or_create.append(song)

            # Commit the session if there were any changes
            db.session.commit()

        return tracks_to_update_or_create
    else:
        logging.error(f"get_top_tracks: {response.status_code}")
        return []



#testing saved albums

def get_saved_albums(access_token, limit=50):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/albums?limit={limit}", headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        logging.error(f"get_saved_albums: {response.status_code}")
        return []





def get_top_artists(access_token, limit=50):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/top/artists?limit={limit}", headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        logging.error(f"get_top_tracks: {response.status_code}")
        return []


import requests

def create_playlist(access_token, user_id, playlist_name, playlist_description):
    """Create a Spotify playlist for a user."""
    
    # Endpoint for creating a new playlist
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    
    # The headers for the HTTP request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # The payload data for the new playlist
    payload = {
        "name": playlist_name,
        "description": playlist_description,
        "public": False  # or True, if you want the playlist to be public
    }
    
    # Make the post request to create a new playlist
    response = requests.post(url, headers=headers, json=payload)
    
    # Check for successful response
    if response.status_code == 201:
        return response.json()  # Return the new playlist data as JSON
    else:
        return None  # Or handle error responses differently

def add_tracks_to_playlist(access_token, playlist_id, track_uris):
    """Add tracks to a Spotify playlist."""
    
    # Endpoint for adding tracks to a playlist
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    
    # The headers for the HTTP request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # The payload data for the tracks to add
    payload = {
        "uris": track_uris
    }
    
    # Make the post request to add tracks
    response = requests.post(url, headers=headers, json=payload)
    
    # Check for successful response
    if response.status_code in (200, 201):
        return response.json()  # Return the response data as JSON
    else:
        return None  # Or handle error responses differently

    
def fetch_us_top_hits_features(headers):
    response = requests.get(f"https://api.spotify.com/v1/playlists/{US_TOP_HITS_PLAYLIST_ID}/tracks", headers=headers)
    us_top_hits_tracks = response.json()
    us_top_hits_track_ids = [track['track']['id'] for track in us_top_hits_tracks['tracks']['items']]
    features_response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={','.join(us_top_hits_track_ids)}", headers=headers)
    return features_response.json(), us_top_hits_tracks
def get_user_playlists(access_token, user_id, limit=50):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/playlists?limit={limit}", headers=headers)

    if response.status_code == 200:
        playlists_data = response.json()['items']
        for playlist_data in playlists_data:
            playlist_id = playlist_data['id']
            # Check if playlist exists in the database
            existing_playlist = Playlist.query.filter_by(id=playlist_id).first()
            if not existing_playlist:
                # Add the new playlist to the database
                new_playlist = Playlist(
                    id=playlist_id,
                    name=playlist_data['name'],
                    user_id=user_id  # Make sure user_id is a string if your db model expects a string
                )
                db.session.add(new_playlist)
        db.session.commit()  # Commit once after all additions
        
        # Fetch playlists from the database
        db_playlists = Playlist.query.filter_by(user_id=user_id).all()
        return [{'id': playlist.id, 'name': playlist.name} for playlist in db_playlists]
    else:
        logging.error(f"get_user_playlists: {response.status_code}")
        return []

def get_playlist_tracks(access_token, user_id, playlist_id):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
    
    if response.status_code == 200:
        tracks_data = response.json()['items']
        for track_item in tracks_data:
            track = track_item['track']
            track_id = track['id']
            # Check if song exists in the database
            existing_song = Song.query.filter_by(id=track_id, uid=user_id).first()
            if not existing_song:
                # Add the new song to the database
                new_song = Song(
                    uid=user_id,
                    id=track_id,
                    name=track['name'],
                    artists=', '.join(artist['name'] for artist in track['artists']),
                    playlist_id=playlist_id
                )
                db.session.add(new_song)
        db.session.commit()  # Commit the changes after processing all tracks

        # Fetch songs from the database
        db_songs = Song.query.filter_by(playlist_id=playlist_id, uid=user_id).all()
        return [{
            'id': song.id,
            'name': song.name,
            'artists': song.artists
        } for song in db_songs]
    else:
        logging.error(f"get_playlist_tracks: {response.status_code}")
        return []
def get_music_characteristics(access_token, song_id):
    # Check if song characteristics exist in the database
    existing_characteristics = SongCharacteristic.query.filter_by(id=song_id).first()
    
    if existing_characteristics:
        # Convert the model instance to a dictionary
        characteristics = {column.name: getattr(existing_characteristics, column.name) 
                           for column in existing_characteristics.__table__.columns}
        return characteristics
    else:
        # Call Spotify API to get the song characteristics
        headers = {'Authorization': f"Bearer {access_token}"}
        response = requests.get(f"https://api.spotify.com/v1/audio-features/{song_id}", headers=headers)

        if response.status_code == 200:
            # Add the new characteristics to the database
            characteristics_data = response.json()
            new_characteristics = SongCharacteristic(
                id=song_id,
                danceability=characteristics_data['danceability'],
                energy=characteristics_data['energy'],
                key=characteristics_data['key'],
                loudness=characteristics_data['loudness'],
                mode=characteristics_data['mode'],
                speechiness=characteristics_data['speechiness'],
                acousticness=characteristics_data['acousticness'],
                instrumentalness=characteristics_data['instrumentalness'],
                liveness=characteristics_data['liveness'],
                valence=characteristics_data['valence'],
                tempo=characteristics_data['tempo']
            )
            db.session.add(new_characteristics)
            db.session.commit()

            # Return the characteristics
            return characteristics_data
        else:
            logging.error(f"get_music_characteristics: {response.status_code}")
            return {}


def get_k_nearest_songs(df_my_tracks, df_us_top_hits, selected_features, min_songs=None, max_songs=None):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    # Preprocess the data
    scaler = StandardScaler().fit(df_my_tracks[selected_features])
    X_my_tracks = scaler.transform(df_my_tracks[selected_features])
    X_us_top_hits = scaler.transform(df_us_top_hits[selected_features])

    # Fit the KNN model
    knn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X_us_top_hits)
    distances, indices = knn.kneighbors(X_my_tracks)

    # Flatten the arrays and get unique song indices
    flat_distances = distances.flatten()
    flat_indices = indices.flatten()

    # Sort the distances and indices together
    sorted_recommendations = sorted(zip(flat_distances, flat_indices), key=lambda x: x[0])
    
    # If a maximum number of songs is specified, get the top closest songs
    if max_songs is not None:
        sorted_recommendations = sorted_recommendations[:max_songs]

    # If a minimum number of songs is specified, ensure at least that many unique songs
    unique_song_indices = []
    for dist, idx in sorted_recommendations:
        if idx not in unique_song_indices:
            unique_song_indices.append(idx)
        if min_songs is not None and len(unique_song_indices) >= min_songs:
            break

    # Extract the recommended unique songs from the US top hits dataframe
    recommended_songs = df_us_top_hits.iloc[unique_song_indices]
    return recommended_songs
def blend_playlists(df_playlist1, df_playlist2, selected_features, num_songs=None):
    # Combine the two playlists
    combined_df = pd.concat([df_playlist1, df_playlist2])

    # Normalize the features of the combined dataframe
    scaler = StandardScaler()
    combined_features = scaler.fit_transform(combined_df[selected_features])

    # Use NearestNeighbors or a clustering algorithm
    knn = NearestNeighbors(n_neighbors=5, metric='cosine')
    knn.fit(combined_features)

    # Find the nearest neighbors within the combined set
    distances, indices = knn.kneighbors(combined_features)

    # Initialize the blended playlist
    blended_playlist = []

    # Set a maximum number of songs from each playlist if not provided
    num_songs = num_songs or min(len(df_playlist1), len(df_playlist2))

    # Loop over indices and distances to build the blended playlist
    for idx, dist in zip(indices, distances):
        # Select songs alternating between playlists
        for i in idx:
            song = combined_df.iloc[i]
            origin_playlist = 'playlist1' if i < len(df_playlist1) else 'playlist2'

            # Check if the song is not already in the blended playlist and if the origin playlist is alternating
            if song['id'] not in [s['id'] for s in blended_playlist] and not (blended_playlist and blended_playlist[-1]['origin'] == origin_playlist):
                song['origin'] = origin_playlist
                blended_playlist.append(song)
                if len(blended_playlist) >= num_songs:
                    break
        if len(blended_playlist) >= num_songs:
            break

    # Convert the blended playlist to a DataFrame
    df_blended_playlist = pd.DataFrame(blended_playlist)

    return df_blended_playlist


@app.route('/')
def index():
    authorized = 'access_token' in session
    username = None
    if 'user_id' in session:
        user = User.query.get(session['user_id'])
        if user:
            username = user.username
    return render_template('index.html', authorized=authorized, username=username)



@app.route('/authorize')
def authorize():
    client_id = app.config['CLIENT_ID']
    redirect_uri = app.config['REDIRECT_URI']
    scope = app.config['SCOPE']
    state_key = os.urandom(15).hex()  # Creating a random state key
    session['state_key'] = state_key

    authorize_url = 'https://accounts.spotify.com/en/authorize?'
    params = {'response_type': 'code', 'client_id': client_id,
              'redirect_uri': redirect_uri, 'scope': scope,
              'state': state_key}
    query_params = urlencode(params)
    response = make_response(redirect(authorize_url + query_params))
    return response

@app.route('/callback/')
def callback():
    # Check for error
    error = request.args.get('error')
    if error:
        # Handle the error based on your requirements
        logging.error(f"Error during authorization: {error}")
        return render_template('error.html', error=error)

    # Check state to prevent CSRF
    state = request.args.get('state')
    if not state or state != session.get('state_key'):
        logging.error("State mismatch error")
        return render_template('error.html', error="State mismatch error")

    # Get the code from the callback URL
    code = request.args.get('code')

    # Use the code to get the access token
    access_token, refresh_token, expires_in = getToken(code)

    # Store the access token, refresh token, and expiration in the session or database
    session['access_token'] = access_token
    session['refresh_token'] = refresh_token
    session['expires_in'] = expires_in

    # Once access token is obtained, get user's Spotify display name
    headers = {'Authorization': f"Bearer {session['access_token']}"}
    response = requests.get("https://api.spotify.com/v1/me", headers=headers)
    if response.status_code == 200:
        spotify_data = response.json()
        spotify_id = spotify_data['id']  # Spotify ID to be used as primary key
        spotify_username = spotify_data['display_name']

    # Check if the user already exists in the database using the Spotify ID
        user = User.query.filter_by(id=spotify_id).first()
        if not user:
        # Create a new user with Spotify ID as the primary key
            user = User(id=spotify_id, username=spotify_username)
            db.session.add(user)
            db.session.commit()

    # Store the Spotify ID in the session
        session['user_id'] = spotify_id
    return redirect(url_for('index'))
@app.route('/recommendations/')
def get_recommendations():
    user_id = session.get('user_id')
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))
    
    get_top_tracks(user_id,access_token, limit=50)
    top_track_ids = [track.id for track in Song.query.with_entities(Song.id).filter_by(uid=user_id,top_tracks=True).all()]


    headers = {'Authorization': f"Bearer {access_token}"}
    features_response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={','.join(top_track_ids)}", headers=headers)
    my_tracks_features = features_response.json()

    us_top_hits_features, us_top_hits_tracks = fetch_us_top_hits_features(headers)

    my_tracks_df = pd.DataFrame(my_tracks_features['audio_features'])
    us_top_hits_df = pd.DataFrame(us_top_hits_features['audio_features'])

    selected_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    
    # Use the new get_k_nearest_songs function to find recommendations
    recommendations_df = get_k_nearest_songs(my_tracks_df, us_top_hits_df, selected_features,min_songs=10, max_songs=10)

    # Get the indices of the recommended songs
    recommended_indices = recommendations_df.index.tolist()

    # Retrieve the recommended songs from the US top hits tracks
    top_songs = [us_top_hits_tracks['tracks']['items'][i]['track'] for i in recommended_indices]
    top_songs_info = [f"{track['name']} by {track['artists'][0]['name']}" for track in top_songs]

    return render_template('recommended_tracks.html', tracks=top_songs_info)


@app.route('/playlists/')
def display_playlists():
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    user_id = session.get('user_id')  # Get the user_id from the session
    if not user_id:
        # Handle the case where there is no user_id in the session
        return redirect(url_for('authorize'))  # Replace 'authorize' with your actual authorization route

    playlists = get_user_playlists(access_token, user_id)
    
    # Get all playlists for this user from the database (now they should be updated)
    user_playlists = Playlist.query.filter_by(user_id=user_id).all()

    return render_template('display_playlists.html', playlists=user_playlists)


@app.route('/playlists/<playlist_id>/tracks/')
def display_tracks(playlist_id):
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    user_id = session.get('user_id')  # Get the user_id from the session
    if not user_id:
        # Handle the case where there is no user_id in the session
        return redirect(url_for('authorize'))  # Replace 'authorize' with your actual authorization route

    tracks = get_playlist_tracks(access_token, user_id, playlist_id)
    
    # Get all songs for this playlist from the database (now they should be updated)
    playlist_songs = Song.query.filter_by(playlist_id=playlist_id).all()
    return render_template('display_tracks.html', tracks=tracks, playlist_songs=playlist_songs)



@app.route('/top_artists/')
def display_artists():
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))
    artists = get_top_artists(access_token)

    for artist in artists:
        artist_name = artist['name']
        artist_id = artist['id']
        user = session['user_id']

        existing_artist = Top_Artists.query.filter_by(uid=user, id=artist_id).first()
        if existing_artist is None:
            top_artist = Top_Artists(uid=user, id=artist_id, name=artist_name)
            db.session.add(top_artist)
            db.session.commit()

    top_artists = Top_Artists.query.filter_by(uid=user).all()

    return render_template('top_artists.html', artists=artists)

@app.route('/saved_albums/')
def display_saved_albums():
    #TESTING DON'T TOUCH
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    saved_albums = get_saved_albums(access_token)

    for album in saved_albums:
        print(album)

    return render_template('saved_albums.html', saved_albums = saved_albums)

@app.route('/top_tracks/')
def display_top_tracks():
    access_token = session.get('access_token')
    user_id = session.get('user_id')
    # Redirect to authorization if there's no access token
    if not access_token or not user_id:
        return redirect(url_for('authorize'))

    # Retrieve and/or store top tracks in the database
    # Assuming get_top_tracks handles the insertion/update of tracks
    get_top_tracks(user_id, access_token)

    # Now query the database for top tracks for the current user
    # Corrected filter_by usage
    top_tracks = Song.query.filter_by(uid=user_id, top_tracks=True).all()
    # Render the top_tracks page and pass the retrieved top tracks to the template
    return render_template('top_tracks.html', top_tracks=top_tracks)

@app.route('/search', methods=['POST'])
def search():

    search_query = request.form.get('search-query')
    # Query the database for tracks with titles matching the search_query
    user = session['user_id']
    results = Song.query.filter(Song.name.like(f'{search_query}%'), Song.uid == user).all()
    return render_template('results.html', results=results)

@app.route('/blend_playlists', methods=['GET', 'POST'])
def blend_playlists_route():
    # Assuming you have a way to get the user's access token and user_id
    access_token = session.get('access_token')
    user_id = session.get('user_id')

    # Make sure the user is authenticated and we have the access token
    if access_token is None or user_id is None:
        return redirect(url_for('login'))  # Redirect to login if not authenticated

    if request.method == 'POST':
        # When the user submits the form, you will receive the two selected playlist IDs
        playlist_id1 = request.form.get('playlist1')
        playlist_id2 = request.form.get('playlist2')
        
        # Fetch tracks for both playlists
        tracks_playlist1 = get_playlist_tracks(access_token, user_id, playlist_id1)
        tracks_playlist2 = get_playlist_tracks(access_token, user_id, playlist_id2)

        # Convert track data to dataframes
        df_playlist1_tracks = pd.DataFrame(tracks_playlist1)
        df_playlist2_tracks = pd.DataFrame(tracks_playlist2)

        # Get music characteristics for each track and add them to the dataframe
        df_playlist1_chars = pd.DataFrame([get_music_characteristics(access_token, track['id']) for track in tracks_playlist1])
        df_playlist2_chars = pd.DataFrame([get_music_characteristics(access_token, track['id']) for track in tracks_playlist2])

        # Merge the tracks info with their characteristics
        df_playlist1 = pd.merge(df_playlist1_tracks, df_playlist1_chars, on='id')
        df_playlist2 = pd.merge(df_playlist2_tracks, df_playlist2_chars, on='id')

        # Define the features we are interested in for blending
        selected_features = [
            'danceability', 'energy', 'key', 'loudness',
            'mode', 'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo'
        ]

        # Use the blend_playlists function to find closest matches
        blended_playlist_df = blend_playlists(df_playlist1, df_playlist2, selected_features)
        session['blended_tracks_ids'] = blended_playlist_df['id'].tolist()

        # Convert the blended tracks to a list of dictionaries to be easily rendered or used
        blended_tracks_list = blended_playlist_df.to_dict('records')

        # Render the blended playlist in the template
        return render_template('show_blended_playlist.html', blended_tracks=blended_tracks_list)

    # If it's a GET request, show the playlists to the user to select which to blend
    user_playlists = get_user_playlists(access_token, user_id)
    return render_template('blend_playlists.html', playlists=user_playlists)
@app.route('/create_playlist', methods=['POST'])
def create_playlist_route():
    access_token = session.get('access_token')
    user_id = session.get('user_id')

    if access_token is None or user_id is None:
        return redirect(url_for('login'))

    playlist_name = request.form.get('playlist_name')
    if playlist_name:
        # Retrieve the track IDs from the session or database
        blended_tracks_ids = session.get('blended_tracks_ids')  # This should be a list of track IDs

        # Create the new playlist on Spotify
        playlist_id = create_spotify_playlist(access_token, user_id, playlist_name)
        
        # Add tracks to the new playlist
        if playlist_id:
            add_tracks_to_playlist(access_token, playlist_id, blended_tracks_ids)

        return render_template('playlist_created.html', playlist_id=playlist_id)
    else:
        return "No playlist name provided", 400

# Helper function to create a new playlist
def create_spotify_playlist(access_token, user_id, playlist_name):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }

    payload = {
        'name': playlist_name,
        'description': 'New playlist description',
        'public': False  # or True, depending on your preference
    }

    response = requests.post(
        f'https://api.spotify.com/v1/users/{user_id}/playlists',
        headers=headers,
        json=payload
    )

    if response.status_code == 201:
        return response.json()['id']
    else:
        # Log the error or handle it accordingly
        print(f'Error creating playlist: {response.json()}')
        return None


# Helper function to add tracks to the new playlist
def add_tracks_to_playlist(access_token, playlist_id, track_ids):
    headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
    # Spotify API expects a list of track URIs, not IDs
    track_uris = [f'spotify:track:{track_id}' for track_id in track_ids]
    data = json.dumps({'uris': track_uris})
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    response = requests.post(url, headers=headers, data=data)
    
    return response.ok

@app.route('/blend_users', methods=['GET', 'POST'])
def blend_users():
    current_user_id = session.get('user_id')  # Assume we store the current user's ID in the session
    access_token = session.get('access_token')  # And the access token

    # Check if the user is authenticated
    if not access_token or not current_user_id:
        return redirect(url_for('login'))  # Redirect to login if not authenticated

    if request.method == 'POST':
        # Get the chosen user's ID from the form
        chosen_user_id = request.form.get('chosen_user_id')

        # Fetch the top tracks for the current user
        get_top_tracks(current_user_id, access_token, limit=50)
        current_user_top_tracks = Song.query.filter_by(uid=current_user_id, top_tracks=True).all()
        
        # Fetch the top tracks for the chosen user
        chosen_user_top_tracks = Song.query.filter_by(uid=chosen_user_id, top_tracks=True).all()
        # Get music characteristics for each track from both users
        current_user_tracks_features = [get_music_characteristics(access_token, track.id) for track in current_user_top_tracks]
        chosen_user_tracks_features = [get_music_characteristics(access_token, track.id) for track in chosen_user_top_tracks]

        # Convert to dataframes
        df_current_user_tracks = pd.DataFrame(current_user_tracks_features)
        df_chosen_user_tracks = pd.DataFrame(chosen_user_tracks_features)

        # Define the features we are interested in for blending
        selected_features = [
            'danceability', 'energy', 'key', 'loudness',
            'mode', 'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo'
        ]

        # Assuming you have a blend_tracks function that finds the blend based on the selected features
        blended_tracks_df =blend_playlists(df_current_user_tracks, df_chosen_user_tracks, selected_features)
        # You can store blended track IDs in the session or database as needed
        session['blended_tracks_ids'] = blended_tracks_df['id'].tolist()
        
        song_ids = blended_tracks_df['id'].tolist()

# Query the Song table to get the songs by their IDs
        songs_info = Song.query.filter(Song.id.in_(song_ids)).all()

        songs_info_df = pd.DataFrame([{
        'id': song.id,
        'name': song.name,
        'artists': song.artists
        } for song in songs_info])

        blended_tracks_complete_df = pd.merge(blended_tracks_df, songs_info_df, on='id')
        blended_tracks_unique_df = blended_tracks_complete_df.drop_duplicates(subset='id', keep='first')

        blended_tracks_list = blended_tracks_unique_df.to_dict('records')

        
        return render_template('show_blended_playlist.html', blended_tracks=blended_tracks_list)

    # If it's a GET request, show the user a form where they can choose another user to blend with
    other_users = User.query.filter(User.id != current_user_id).all()
    return render_template('choose_user_to_blend.html', users=other_users)






if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
/var/folders/hf/sltcsxm544n8h8kgwt1k0fsm0000gn/T/ipykernel_754/834235335.py:421: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = User.query.get(session['user_id'])
[2023-11-09 20:34:05,856] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/Users/dungnguyen/csharp/.venv/lib/python3.11/site-packages/flask/app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/dungnguyen/csharp/.venv/lib/python3.11/sit

{'added_at': '2023-07-29T21:00:43Z', 'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/12j6dJrPXanCBwY599pZxf'}, 'href': 'https://api.spotify.com/v1/artists/12j6dJrPXanCBwY599pZxf', 'id': '12j6dJrPXanCBwY599pZxf', 'name': 'Matt Maltese', 'type': 'artist', 'uri': 'spotify:artist:12j6dJrPXanCBwY599pZxf'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'B

127.0.0.1 - - [09/Nov/2023 20:34:55] "GET /saved_albums/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2023 20:34:55] "GET /saved_albums/ HTTP/1.1" 200 -


{'added_at': '2023-07-29T21:00:43Z', 'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/12j6dJrPXanCBwY599pZxf'}, 'href': 'https://api.spotify.com/v1/artists/12j6dJrPXanCBwY599pZxf', 'id': '12j6dJrPXanCBwY599pZxf', 'name': 'Matt Maltese', 'type': 'artist', 'uri': 'spotify:artist:12j6dJrPXanCBwY599pZxf'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'B

In [50]:
import sqlite3

# Replace with your actual database file path
database_path ='/home/cars327/csharp_final/csharp/db/User.db'
user_id = 'izkud41eox5ydye21ce50q9qw'  # Replace with the actual user ID

# Connect to the SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# SQL query that represents the same logic as the ORM query above
query = """
SELECT * FROM Song
WHERE uid = ? AND playlist_id = 'Top_Tracks'
"""

# Execute the query, passing the user_id as a parameter to prevent SQL injection
cursor.execute(query, (user_id,))

# Fetch all the resulting rows
top_tracks = cursor.fetchall()

# Print the rows, or process them as needed
for track in top_tracks:
    print(track)

# Close the connection
conn.close()


('izkud41eox5ydye21ce50q9qw', '02o33KMvrYSHs2hLM03d00', 'Рандеву', 'Soltwine', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '05vvPTidbjvDySLc9o65YS', "Don't Forget About Me", 'd4vd', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0CIJzHUsNI99AtMKOrEcYA', 'Io ti darò di più', 'Memo Remigi', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0EF1EE8zusg3Y869e56JFd', "Nothing's New", 'Rio Romeo', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0MqAcZlEeS5q152ehuFDTr', 'ME FIRST', 'Kari Faux', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0RBw4ODUQPO4cuAOZtBGga', 'Hurts Me', 'Tory Lanez, Trippie Redd, Yoko Gold', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0T5iIrXA4p5GsubkhuBIKV', 'Until I Found You', 'Stephen Sanchez', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0UpZJ1hUK4q1yzjQzXI32B', 'Endless No Mores - A COLORS SHOW', 'Elmiene', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0V5cvmTKsYmF5FmGGEAfmS', 'Je te laisserai des mots', 'Patrick Watson', 'Top_Tracks')
('izkud41eox5ydye21ce50q9qw', '0WNfQxDG